데이터 전처리 과정 분석

convert_h36m.py 코드를 통해 h36m_sh_conf_cam_source_final.pkl 파일로부터 clip들을 나누어 data/motion3d/MB3D_f243s81/H36M-SH 폴더에 저장함
그 과정에서 전처리 과정을 거쳐 원본 파일과 새로 생성된 파일의 값이 달라짐 -> 이 부분을 이해해야 AI Hub 데이터를 동일한 형식으로 만들어 훈련시킬 수 있음

In [1]:
import os
import pickle
import copy
import numpy as np
from tqdm import tqdm

# 원본 파일
with open('../../data/motion3d/h36m_sh_conf_cam_source_final.pkl', 'rb') as f:
	datareader = pickle.load(f)

### h36m_sh_conf_cam_source_final.pkl 분석

In [2]:
datareader.keys()

dict_keys(['train', 'test'])

In [3]:
datareader['train'].keys()

dict_keys(['joint_2d', 'confidence', 'joint3d_image', 'camera_name', 'source'])

In [4]:
datareader['train']['joint_2d'].shape # (1559752, 17, 2) array
datareader['train']['confidence'].shape # (1559752, 17, 1) array
datareader['train']['joint3d_image'].shape # (1559752, 17, 3)
datareader['train']['camera_name'].shape # (1559752,) array
len(datareader['train']['source']) # 1559752 list

1559752

In [5]:
1559752/5

311950.4

In [6]:
set(datareader['train']['source']) # s01, s05, s06, s07, s08 / act02 ~ act16 / cam01 ~ cam04

{'s_01_act_02_cam_01',
 's_01_act_02_cam_02',
 's_01_act_02_cam_03',
 's_01_act_02_cam_04',
 's_01_act_03_cam_01',
 's_01_act_03_cam_02',
 's_01_act_03_cam_03',
 's_01_act_03_cam_04',
 's_01_act_04_cam_01',
 's_01_act_04_cam_02',
 's_01_act_04_cam_03',
 's_01_act_04_cam_04',
 's_01_act_05_cam_01',
 's_01_act_05_cam_02',
 's_01_act_05_cam_03',
 's_01_act_05_cam_04',
 's_01_act_06_cam_01',
 's_01_act_06_cam_02',
 's_01_act_06_cam_03',
 's_01_act_06_cam_04',
 's_01_act_07_cam_01',
 's_01_act_07_cam_02',
 's_01_act_07_cam_03',
 's_01_act_07_cam_04',
 's_01_act_08_cam_01',
 's_01_act_08_cam_02',
 's_01_act_08_cam_03',
 's_01_act_08_cam_04',
 's_01_act_09_cam_01',
 's_01_act_09_cam_02',
 's_01_act_09_cam_03',
 's_01_act_09_cam_04',
 's_01_act_10_cam_01',
 's_01_act_10_cam_02',
 's_01_act_10_cam_03',
 's_01_act_10_cam_04',
 's_01_act_11_cam_01',
 's_01_act_11_cam_02',
 's_01_act_11_cam_03',
 's_01_act_11_cam_04',
 's_01_act_12_cam_01',
 's_01_act_12_cam_02',
 's_01_act_12_cam_03',
 's_01_act_

In [7]:
set(datareader['test']['source']) # s09, s11 / act02 ~ act16 / cam01 ~ cam04

{'s_09_act_02_subact_01_ca_01',
 's_09_act_02_subact_01_ca_02',
 's_09_act_02_subact_01_ca_03',
 's_09_act_02_subact_01_ca_04',
 's_09_act_02_subact_02_ca_01',
 's_09_act_02_subact_02_ca_02',
 's_09_act_02_subact_02_ca_03',
 's_09_act_02_subact_02_ca_04',
 's_09_act_03_subact_01_ca_01',
 's_09_act_03_subact_01_ca_02',
 's_09_act_03_subact_01_ca_03',
 's_09_act_03_subact_01_ca_04',
 's_09_act_03_subact_02_ca_01',
 's_09_act_03_subact_02_ca_02',
 's_09_act_03_subact_02_ca_03',
 's_09_act_03_subact_02_ca_04',
 's_09_act_04_subact_01_ca_01',
 's_09_act_04_subact_01_ca_02',
 's_09_act_04_subact_01_ca_03',
 's_09_act_04_subact_01_ca_04',
 's_09_act_04_subact_02_ca_01',
 's_09_act_04_subact_02_ca_02',
 's_09_act_04_subact_02_ca_03',
 's_09_act_04_subact_02_ca_04',
 's_09_act_05_subact_01_ca_01',
 's_09_act_05_subact_01_ca_02',
 's_09_act_05_subact_01_ca_03',
 's_09_act_05_subact_01_ca_04',
 's_09_act_05_subact_02_ca_01',
 's_09_act_05_subact_02_ca_02',
 's_09_act_05_subact_02_ca_03',
 's_09_a

In [8]:
datareader['test'].keys()

dict_keys(['joint_2d', 'confidence', 'joint3d_image', 'joints_2.5d_image', '2.5d_factor', 'camera_name', 'action', 'source'])

In [9]:
datareader['test']['joint_2d'].shape # (566920, 17, 2) array
datareader['test']['confidence'].shape # (566920, 17, 1) array
datareader['test']['joint3d_image'].shape # (566920, 17, 3) array
datareader['test']['joints_2.5d_image'].shape # (566920, 17, 3) array = 2.5d_factor * joint3d_image
datareader['test']['2.5d_factor'].shape # (566920,) array
datareader['test']['camera_name'].shape # (566920,) array
len(datareader['test']['action']) # 566920 list
len(datareader['test']['source']) # 566920 list

566920

In [10]:
566920/2/15/30

629.911111111111

In [11]:
datareader['test']['2.5d_factor'][datareader['test']['camera_name'] == '54138969']

array([4.656527 , 4.6566076, 4.6566916, ..., 2.8969798, 2.9066522,
       2.9163694], dtype=float32)

In [12]:
datareader['test']['joint3d_image'][0][0]*datareader['test']['2.5d_factor'][0] == datareader['test']['joints_2.5d_image'][0][0]

array([ True,  True,  True])

In [13]:
datareader['test']['source'][0]

's_09_act_02_subact_01_ca_01'

### aihub.pkl 생성

In [14]:
import os
import getpass

user = getpass.getuser()
alphapose_root = '/home/{}/codes/AlphaPose'.format(user)
motionbert_root = '/home/{}/codes/MotionBERT'.format(user)
aihub_root = '/home/{}/Datasets/HAAI/AIHUB'.format(user)
ap_aihub_result_root = alphapose_root + "/examples/aihub_result"
alphapose_root, motionbert_root, aihub_root, ap_aihub_result_root

('/home/hrai/codes/AlphaPose',
 '/home/hrai/codes/MotionBERT',
 '/home/hrai/Datasets/HAAI/AIHUB',
 '/home/hrai/codes/AlphaPose/examples/aihub_result')

In [15]:
os.chdir(motionbert_root)
from custom_codes.test_utils import *

In [16]:
action_dict = {
    '23' : "BenchPress",
    '30' : "Squat",
    '33' : "Push_Up",
    '49' : "Butterfly"
}

In [17]:
aihub = {'train': {}, 'test': {}}
# aihub['test']['joint_2d'] = None
# aihub['test']['confidence'] = None
# aihub['test']['joint3d_image'] = None
# aihub['test']['joints_2.5d_image'] = None
# aihub['test']['2.5d_factor'] = None
# aihub['test']['camera_name'] = None
# aihub['test']['action'] = None
# aihub['test']['source'] = None

In [18]:
joint_2ds = []
confidences = []
joint3d_images = []
joints_25d_images = []
_25d_factors = []
camera_names = []
actions = []
sources = []
frames = []
world_3ds = []
cam_3ds = []
cam_params = []

In [19]:
from zipfile import ZipFile
aihub_3d_gt_path = os.path.join(motionbert_root, "data/motion3d/AIHUB/train/[라벨]3D_json.zip")
zip_label = ZipFile(os.path.join(aihub_3d_gt_path), 'r')
aihub_camera_param_path = os.path.join(motionbert_root, "data/motion3d/AIHUB/train/Camera_json_train.zip")
zip_param = ZipFile(os.path.join(aihub_camera_param_path), 'r')

In [8]:
from tqdm import tqdm

### train

In [9]:
# whitelist
action_list = []
actor_list = []
cam_list = []

blacklist = ['F170D', 'M170C', 'F160A']

for video in os.listdir(ap_aihub_result_root):
    #print(video)
    _, action, actor, cam_num = video.split('_')
    if len(action_list) > 0:
        if action not in action_list:
            continue
    if len(actor_list) > 0:
        if actor not in actor_list:
            continue
    if len(cam_list) > 0:
        if cam_num not in cam_list:
            continue
    if actor in blacklist:
        print(actor, 'is in blacklist')
        continue
    
    print(actor, cam_num, action)
    video_root = os.path.join(ap_aihub_result_root, video)
    print(video_root)
    print(os.listdir(video_root))
    ap_result_json = os.path.join(video_root, 'alphapose-results.json')
    ap_result_video = os.path.join(video_root, 'test.mp4')
    if not os.path.exists(ap_result_json):
        print('no json')
        continue
    ap_result = readJSON(ap_result_json)
    num_frames = len(ap_result)
    print('ap: ', num_frames)

    idx_list = []
    
    for i in tqdm(range(num_frames)):
        #print(frame.keys()) # dict_keys(['image_id', 'category_id', 'keypoints', 'score', 'idx'])
        frame = ap_result[i]
        frame_num = int(frame['image_id'].split('_')[-1].split('.')[0])
        #print(frame_num)

        # load gt 
        load_path = "3D_json/{}_{}/3D_{}_{}_{}.json".format(action, actor, action, actor, frame_num)
        #print(load_path)
        try:
            gt = json.loads(zip_label.read(load_path).decode('utf-8'))
        except:
            print('no gt')
            continue

        # load camera parameter
        load_path = "Camera_json/train/{}_{}_{}.json".format(action, actor, cam_num)
        try:
            camera_param = json.loads(zip_param.read(load_path).decode('utf-8'))
        except:
            print('no camera param')
            continue
        _, extrinsic_properties, R, T, R_c, C, intrinsic_properties, fx, fy, cx, cy = getAIHubCameraParameter(camera_param)

        ## joint 2d & confidence
        keypoints = np.array(frame['keypoints']).reshape(1,-1, 3)
        # halpe to h36m joint
        input_2d = halpe2h36m(keypoints)[0, :, :] # (17, 2)

        ## joint3d_image
        world_3d = aihub2h36m(np.array(gt['annotations']['3d_pos'])[:, :3].reshape(1, 24, 3))[0]

        # world to camera
        pos = copy.deepcopy(world_3d)
        cam_3d = World2CameraCoordinate(pos, extrinsic_properties) # World coordinate -> Camera coordinate
        cam_3d_hat = get_rootrel_pose(cam_3d)
        
        # camera to image
        box = infer_box(cam_3d, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0)
        img_2d, img_3d = camera_to_image_frame(cam_3d, box, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0) 
        #img_2d_hat = get_rootrel_pose(img_2d) # (17, 2) # root-relative pose 
        img_3d_hat = get_rootrel_pose(img_3d) # (17, 3) # root-relative pose 

        # 2.5d factor
        pred_lambda = optimize_scaling_factor(img_3d_hat, cam_3d_hat) # x,y,z 사용

        # joint 2.5d image
        img_25d = img_3d * pred_lambda
        img_25d_hat = get_rootrel_pose(img_25d)

        joint_2ds.append(input_2d[:, :2])
        confidences.append(input_2d[:, 2])
        joint3d_images.append(img_3d)
        joints_25d_images.append(img_25d)
        _25d_factors.append(pred_lambda)
        camera_names.append(cam_num)
        actions.append(action)
        sources.append(video)
        frames.append(frame_num)
        world_3ds.append(world_3d)
        cam_3ds.append(cam_3d)
        cam_params.append(camera_param)
    
    #break


M160C 6 31
/home/hrai/codes/AlphaPose/examples/aihub_result/res_31_M160C_6
['alphapose-results.json', 'vis']
ap:  220


  0%|          | 0/220 [00:00<?, ?it/s]

100%|██████████| 220/220 [00:04<00:00, 48.00it/s]


M170A 8 56
/home/hrai/codes/AlphaPose/examples/aihub_result/res_56_M170A_8
['vis']
no json
F170A 7 52
/home/hrai/codes/AlphaPose/examples/aihub_result/res_52_F170A_7
['vis']
no json
M160C 3 23
/home/hrai/codes/AlphaPose/examples/aihub_result/res_23_M160C_3
['vis']
no json
M160B 1 35
/home/hrai/codes/AlphaPose/examples/aihub_result/res_35_M160B_1
['alphapose-results.json', 'vis']
ap:  1080


100%|██████████| 1080/1080 [00:20<00:00, 51.52it/s]


...

 12%|█▏        | 57/495 [00:00<00:04, 99.86it/s]

no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt
no gt


100%|██████████| 495/495 [00:08<00:00, 55.58it/s]


M180B 8 23
/home/hrai/codes/AlphaPose/examples/aihub_result/res_23_M180B_8
['alphapose-results.json', 'vis']
ap:  435


100%|██████████| 435/435 [00:08<00:00, 54.04it/s]


In [10]:
aihub['train']['joint_2d'] = copy.deepcopy(np.array(joint_2ds))
aihub['train']['confidence'] = copy.deepcopy(np.array(confidences))
aihub['train']['joint3d_image'] = copy.deepcopy(np.array(joint3d_images))
aihub['train']['joints_2.5d_image'] = copy.deepcopy(np.array(joints_25d_images))
aihub['train']['2.5d_factor'] = copy.deepcopy(np.array(_25d_factors))
aihub['train']['camera_name'] = copy.deepcopy(np.array(camera_names))
aihub['train']['action'] = copy.deepcopy(np.array(actions))
aihub['train']['source'] = copy.deepcopy(np.array(sources))
aihub['train']['frame'] = copy.deepcopy(np.array(frames))
aihub['train']['world_3d'] = copy.deepcopy(np.array(world_3ds))
aihub['train']['cam_3d'] = copy.deepcopy(np.array(cam_3ds))
aihub['train']['cam_param'] = copy.deepcopy(np.array(cam_params))

pwd

### test

In [11]:
joint_2ds = []
confidences = []
joint3d_images = []
joints_25d_images = []
_25d_factors = []
camera_names = []
actions = []
sources = []
frames = []
world_3ds = []
cam_3ds = []
cam_params = []

In [12]:
# whitelist
action_list = ['30']
actor_list = ['F170D', 'M170C', 'F160A']
cam_list = []

blacklist = []

for video in os.listdir(ap_aihub_result_root):
    #print(video)
    _, action, actor, cam_num = video.split('_')
    if action not in action_list:
        continue
    if len(actor_list) > 0:
        if actor not in actor_list:
            continue
    if len(cam_list) > 0:
        if cam_num not in cam_list:
            continue
    if actor in blacklist:
        print(actor, 'is in blacklist')
        continue
    
    print(actor, cam_num, action)
    video_root = os.path.join(ap_aihub_result_root, video)
    print(video_root)
    print(os.listdir(video_root))
    ap_result_json = os.path.join(video_root, 'alphapose-results.json')
    ap_result_video = os.path.join(video_root, 'test.mp4')
    if not os.path.exists(ap_result_json):
        print('no json')
        continue
    ap_result = readJSON(ap_result_json)
    num_frames = len(ap_result)
    print('ap: ', num_frames)

    idx_list = []
    
    for i in tqdm(range(num_frames)):
        #print(frame.keys()) # dict_keys(['image_id', 'category_id', 'keypoints', 'score', 'idx'])
        frame = ap_result[i]
        frame_num = int(frame['image_id'].split('_')[-1].split('.')[0])
        #print(frame_num)

        # load gt 
        load_path = "3D_json/{}_{}/3D_{}_{}_{}.json".format(action, actor, action, actor, frame_num)
        #print(load_path)
        try:
            gt = json.loads(zip_label.read(load_path).decode('utf-8'))
        except:
            print('no gt')
            continue

        # load camera parameter
        load_path = "Camera_json/train/{}_{}_{}.json".format(action, actor, cam_num)
        try:
            camera_param = json.loads(zip_param.read(load_path).decode('utf-8'))
        except:
            print('no camera param')
            continue
        _, extrinsic_properties, R, T, R_c, C, intrinsic_properties, fx, fy, cx, cy = getAIHubCameraParameter(camera_param)

        ## joint 2d & confidence
        keypoints = np.array(frame['keypoints']).reshape(1,-1, 3)
        # halpe to h36m joint
        input_2d = halpe2h36m(keypoints)[0, :, :] # (17, 2)

        ## joint3d_image
        world_3d = aihub2h36m(np.array(gt['annotations']['3d_pos'])[:, :3].reshape(1, 24, 3))[0]

        # world to camera
        pos = copy.deepcopy(world_3d)
        cam_3d = World2CameraCoordinate(pos, extrinsic_properties) # World coordinate -> Camera coordinate
        cam_3d_hat = get_rootrel_pose(cam_3d)
        
        # camera to image
        box = infer_box(cam_3d, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0)
        img_2d, img_3d = camera_to_image_frame(cam_3d, box, {'fx': fx, 'fy': fy, 'cx': cx, 'cy': cy}, 0) 
        #img_2d_hat = get_rootrel_pose(img_2d) # (17, 2) # root-relative pose 
        img_3d_hat = get_rootrel_pose(img_3d) # (17, 3) # root-relative pose 

        # 2.5d factor
        pred_lambda = optimize_scaling_factor(img_3d_hat, cam_3d_hat) # x,y,z 사용

        # joint 2.5d image
        img_25d = img_3d * pred_lambda
        img_25d_hat = get_rootrel_pose(img_25d)

        joint_2ds.append(input_2d[:, :2])
        confidences.append(input_2d[:, 2])
        joint3d_images.append(img_3d)
        joints_25d_images.append(img_25d)
        _25d_factors.append(pred_lambda)
        camera_names.append(cam_num)
        actions.append(action)
        sources.append(video)
        frames.append(frame_num)
        world_3ds.append(world_3d)
        cam_3ds.append(cam_3d)
        cam_params.append(camera_param)
    
    #break


F160A 1 30
/home/hrai/codes/AlphaPose/examples/aihub_result/res_30_F160A_1
['alphapose-results.json', 'vis']
ap:  885


 14%|█▍        | 125/885 [00:02<00:14, 52.15it/s]

100%|██████████| 885/885 [00:16<00:00, 52.29it/s]


F160A 8 30
/home/hrai/codes/AlphaPose/examples/aihub_result/res_30_F160A_8
['alphapose-results.json', 'vis']
ap:  929


100%|██████████| 929/929 [00:17<00:00, 52.32it/s]


...

100%|██████████| 225/225 [00:04<00:00, 52.21it/s]


F170D 5 30
/home/hrai/codes/AlphaPose/examples/aihub_result/res_30_F170D_5
['alphapose-results.json', 'vis']
ap:  225


100%|██████████| 225/225 [00:04<00:00, 52.21it/s]


F170D 8 30
/home/hrai/codes/AlphaPose/examples/aihub_result/res_30_F170D_8
['alphapose-results.json', 'vis']
ap:  225


100%|██████████| 225/225 [00:04<00:00, 52.68it/s]


In [13]:
aihub['test']['joint_2d'] = copy.deepcopy(np.array(joint_2ds))
aihub['test']['confidence'] = copy.deepcopy(np.array(confidences))
aihub['test']['joint3d_image'] = copy.deepcopy(np.array(joint3d_images))
aihub['test']['joints_2.5d_image'] = copy.deepcopy(np.array(joints_25d_images))
aihub['test']['2.5d_factor'] = copy.deepcopy(np.array(_25d_factors))
aihub['test']['camera_name'] = copy.deepcopy(np.array(camera_names))
aihub['test']['action'] = copy.deepcopy(np.array(actions))
aihub['test']['source'] = copy.deepcopy(np.array(sources))
aihub['test']['frame'] = copy.deepcopy(np.array(frames))
aihub['test']['world_3d'] = copy.deepcopy(np.array(world_3ds))
aihub['test']['cam_3d'] = copy.deepcopy(np.array(cam_3ds))
aihub['test']['cam_param'] = copy.deepcopy(np.array(cam_params))

In [14]:
with open(motionbert_root + '/data/motion3d/tr_aihub_sport_ts_30.pkl', 'wb') as f:
	pickle.dump(aihub, f, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
aihub['train']['joint_2d'].shape, aihub['test']['joint_2d'].shape

((561195, 17, 2), (3407, 17, 2))

In [18]:
for key in aihub['test'].keys():
    print(key, aihub['test'][key].shape)

joint_2d (3407, 17, 2)
confidence (3407, 17)
joint3d_image (3407, 17, 3)
joints_2.5d_image (3407, 17, 3)
2.5d_factor (3407,)
camera_name (3407,)
action (3407,)
source (3407,)


In [19]:
aihub['test']['source']

array(['res_30_F160A_1', 'res_30_F160A_1', 'res_30_F160A_1', ...,
       'res_30_F170D_8', 'res_30_F170D_8', 'res_30_F170D_8'], dtype='<U14')